In [1]:
# system stuff
import sys
sys.path.append("..")
import datetime

# galpy stuff
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, HernquistPotential
from galpy.actionAngle import estimateDeltaStaeckel, actionAngleStaeckel
from galpy.actionAngle import UnboundError

# auriga stuff
from areposnap.gadget import gadget_readsnap
from areposnap.gadget_subfind import load_subfind

from auriga_basics import *
from auriga_functions import *

# main packages stuff
import numpy as np
from scipy import stats

# python stuff
import random
import copy

# plot stuff
from IPython.display import Image, display
import corner

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Circle
from matplotlib import animation
from matplotlib.colors import LogNorm, LinearSegmentedColormap
from matplotlib import rcParams
from matplotlib import colors

%matplotlib inline

rcParams['font.size'] = 16
rcParams['lines.linewidth'] = 1.5
rcParams['axes.linewidth'] = 1.5
rcParams['ytick.major.width'] = 1.5
rcParams['xtick.major.width'] = 1.5
rcParams['image.cmap'] = 'magma'
rcParams['savefig.bbox'] = 'tight'
rcParams['savefig.dpi'] = 300
rcParams['savefig.format'] = 'png'
cmap = copy.copy(plt.cm.get_cmap(rcParams['image.cmap']))
cmap.set_bad((0,0,0))

# display stuff
%config InlineBackend.figure_format = 'retina'
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
# Simulation relevant stuff
machine = 'virgo'
if machine == 'magny':
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
    filedir = "/home/extmilan/masterthesis/files/"
    plotdir = "/home/extmilan/masterthesis/plots/"
elif machine == 'mac': 
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
    filedir = "/Users/smilanov/Documents/masterthesis/auriga_files/files/"
    plotdir = "/Users/smilanov/Documents/masterthesis/auriga_files/plots/"
elif machine == 'virgo': 
    basedir = "/virgo/simulations/Auriga/level4_MHD/"
    filedir = "/u/milas/masterthesis/masterproject/files/"
    plotdir = "/u/milas/masterthesis/masterproject/plots/"
    

In [3]:
# load potential parameters
R0_kpc_all, v0_kms_all, a_MND_kpc_all, b_MND_kpc_all, a_NFWH_kpc_all, a_HB_kpc_all, v0_MND_kms_all, v0_NFWH_kms_all, v0_HB_kms_all = np.loadtxt(filedir + 'potential/fitted_potential_params_jan19.txt')

n_MND  = v0_MND_kms_all**2  / v0_kms_all**2
n_NFWH = v0_NFWH_kms_all**2 / v0_kms_all**2
n_HB   = v0_HB_kms_all**2   / v0_kms_all**2

In [4]:
def setup_galpy_potential(a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB, _REFR0_kpc):
    
    #test input:
    if (a_MND_kpc <= 0.) or (b_MND_kpc <= 0.) or (a_NFWH_kpc <= 0.) or (a_HB_kpc <= 0.) \
       or (n_MND <= 0.) or (n_NFWH <= 0.) or (n_HB <= 0.) or (n_MND >= 1.) or (n_NFWH >= 1.) or (n_HB >= 1.):
        raise ValueError('Error in setup_galpy_potential: '+\
                         'The input parameters for the scaling profiles do not correspond to a physical potential.')
    if np.fabs(n_MND + n_NFWH + n_HB - 1.) > 2e-2:
        raise ValueError('Error in setup_galpy_potential: '+\
                         'The sum of the normalization does not add up to 1.')
        
    #trafo to galpy units:
    a_MND  = a_MND_kpc  / _REFR0_kpc
    b_MND  = b_MND_kpc  / _REFR0_kpc
    a_NFWH = a_NFWH_kpc / _REFR0_kpc
    a_HB   = a_HB_kpc   / _REFR0_kpc
    
    #setup potential:
    disk = MiyamotoNagaiPotential(
                a = a_MND,
                b = b_MND,
                normalize = n_MND)
    halo = NFWPotential(
                a = a_NFWH,
                normalize = n_NFWH)
    bulge = HernquistPotential(
                a = a_HB,
                normalize = n_HB) 
     
    return [disk,halo,bulge]

In [6]:
level = 4

startnr = 126
endnr   = 127

colors = ['black', 'black', 'xkcd:hot pink', 'xkcd:indigo blue', 'xkcd:saffron']
 #2, 3, 4
zsignchange = [False, False, True]
excludedisk = [True, True, True]
for prognr in [2, 3, 4]:
    for jj in range(1):
        if zsignchange[jj]: 
            print("exclude particles which change signs while in disk radius")
            infilename = 'auriga/surviving_GC_zsignchange_IDs_prog{}.txt'.format(prognr)
        elif excludedisk[jj]:
            print('exclude particles which are in disk')
            infilename = 'auriga/surviving_GC_wodisk_IDs_prog{}.txt'.format(prognr)
        else:
            print('keep em all')
            infilename = 'auriga/surviving_GC_all_IDs_prog{}.txt'.format(prognr)

        IDs = np.loadtxt(filedir + infilename)
        progcolor = colors[prognr]

        for halo_number in [24]:  # range(1, 31):
            halodir = basedir+"halo_{0}/".format(halo_number)
            snappath = halodir+"output/"

            for snapnr in range(endnr-1, startnr-1, -1):
                i = snapnr - startnr

                print("level   : {0}".format(level))
                print("halo    : {0}".format(halo_number))
                print("snapnr  : {0}".format(snapnr))
                print("basedir : {0}".format(basedir))
                print("halodir : {0}".format(halodir))
                print("snappath: {0}\n".format(snappath))
                s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
                    haloid=0, galradfac=0.1, verbose=True) 

                # Clean negative and zero values of gmet to avoid RuntimeErrors
                # later on (e.g. dividing by zero)
                s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )

                if zsignchange[jj]: 
                    outfilename = 'actions/right_potential/progenitor{}/GCzsignchange_actions_snap_{}_jan19.txt'.format(prognr, snapnr)
                    outplot = 'actions/right_potential/progenitor{}/GCzsignchange_actions_snap_{}.png'.format(prognr, snapnr)
                elif excludedisk[jj]:
                    outfilename = 'actions/right_potential/progenitor{}/GCwodisk_actions_snap_{}_jan19.txt'.format(prognr, snapnr)
                    outplot = 'actions/right_potential/progenitor{}/GCwodisk_actions_snap_{}.png'.format(prognr, snapnr)
                else:
                    outfilename = 'actions/right_potential/progenitor{}/GCall_actions_snap_{}_jan19.txt'.format(prognr, snapnr)
                    outplot = 'actions/right_potential/progenitor{}/GCall_actions_snap_{}.png'.format(prognr, snapnr)

                gcmask = np.isin(s.id, IDs)
                print('Number of GCs: {}.'.format(np.sum(gcmask)))

                print('Sum of normalizations after fitting potential parameters: {}.'.format(np.sum([n_MND[i], n_NFWH[i], n_HB[i]])))

                pot_galpy = setup_galpy_potential(a_MND_kpc_all[i], b_MND_kpc_all[i], a_NFWH_kpc_all[i], a_HB_kpc_all[i], n_MND[i], n_NFWH[i], n_HB[i], R0_kpc_all[i])

                # this would mean that there are no merged GCs
                if np.sum(gcmask) == 0:
                    print("Skipped snap {} since there are no merged GCs.".format(snapnr))
                    #continue


                # get position and velocities of all selected GCs & convert to galpy units
                (R_kpc, phi_rad, z_kpc), (vR_kms, vphi_kms, vz_kms) = get_cylindrical_vectors(s, sf, gcmask)
                # convert physical to galpy units by dividing by REF vals (get velocities from best fit parameters)
                fix_v0_kms = v0_kms_all[i]
                fix_R0_kpc = R0_kpc_all[i]
                R_galpy, vR_galpy, vT_galpy, z_galpy, vz_galpy = R_kpc / fix_R0_kpc, vR_kms / fix_v0_kms, vphi_kms / fix_v0_kms, z_kpc / fix_R0_kpc, vz_kms / fix_v0_kms

                # estimate Delta of the Staeckel potential
                delta = 0.45
                delta = estimateDeltaStaeckel(pot_galpy, R_galpy, z_galpy)
                # CHECK HOW BIG INFLUENCE OF DELTA IS


                # set up the actionAngleStaeckel object
                aAS = actionAngleStaeckel(
                        pot   = pot_galpy,  # potential
                        delta = delta,      # focal length of confocal coordinate system
                        c     = True        # use C code (for speed)
                        )

                jR_galpy, lz_galpy, jz_galpy, r_kpc = np.zeros(len(IDs)), np.zeros(len(IDs)), np.zeros(len(IDs)), np.zeros(len(IDs))
                savedIDs = np.zeros(len(IDs))
                IDs_notworking = []
                for test_i, item in enumerate(IDs):
                    if (test_i % 1000) == 0:
                        print(datetime.datetime.now().time())
                        print(test_i)
                    try: 
                        jR_galpy[test_i], lz_galpy[test_i], jz_galpy[test_i] = aAS(R_galpy[test_i], vR_galpy[test_i], vT_galpy[test_i], z_galpy[test_i], vz_galpy[test_i])
                        r_kpc[test_i] = np.sqrt(R_kpc[test_i]**2 + z_kpc[test_i]**2)
                        savedIDs[test_i] = item
                    except(ValueError, UnboundError):
                        IDs_notworking.append(item)
                        continue

                print('Number of GCs w/o actions:', len(IDs_notworking))
                jR_kpckms, lz_kpckms, jz_kpckms = jR_galpy * fix_R0_kpc * fix_v0_kms, lz_galpy * fix_R0_kpc * fix_v0_kms, jz_galpy * fix_R0_kpc * fix_v0_kms

                # just pick result values of particles of which I actually could calculate actions
                survivor_id_mask = np.isin(IDs, savedIDs)
                survivor_IDs = IDs[survivor_id_mask]
                jR_kpckms, lz_kpckms, jz_kpckms = jR_kpckms[survivor_id_mask], lz_kpckms[survivor_id_mask], jz_kpckms[survivor_id_mask]
                r_kpc = r_kpc[survivor_id_mask]

                save_data = ([survivor_IDs, r_kpc, jR_kpckms, lz_kpckms, jz_kpckms])
                np.savetxt(filedir + outfilename, save_data)

                data = np.vstack([jR_kpckms, lz_kpckms, jz_kpckms])

                if prognr == 2:
                    actionrange = np.array([[0, 10000], [-25000,0], [0,2000]])
                else: actionrange = None
                print(actionrange)
                labels = ['\nJ$_\mathrm{R}$ [kpc km s$^{-1}$]\n', '\nL$_\mathrm{z}$ [kpc km s$^{-1}$]\n', '\nJ$_\mathrm{z}$ [kpc km s$^{-1}$]\n']
                figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = progcolor, range = actionrange, bbox_inches = 'tight')#, range =  [(0.,7500.), (-10000.,10000.),(0., 4000.)])
                figure.savefig(plotdir + outplot)
                plt.show()
                
                

exclude particles which are in disk
level   : 4
halo    : 24
snapnr  : 89
basedir : /virgo/simulations/Auriga/level4_MHD/
halodir : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath: /virgo/simulations/Auriga/level4_MHD/halo_24/output/

Found 1194051 stars.
Rotated pos.
Rotated vel.

galrad  : 0.017763620615005495
redshift: 0.6446418406845371
time    : 0.6080351206337894
center  : [ 0.  0.  0.]

Number of GCs: 24793.
Sum of normalizations after fitting potential parameters: 0.9947238701040797.
21:54:33.839924
0
21:55:50.855627
1000
21:57:13.195687
2000
21:58:33.520998
3000
21:59:51.404629
4000
22:02:07.930209
5000
22:04:28.438421
6000
22:07:28.533003
7000


KeyboardInterrupt: 

In [9]:
zsignchange = [False, True, False]
excludedisk = [False, True, False]

snapnr = 127
for jj in range(1):
    if zsignchange[jj]: 
        infilename = 'actions/right_potential/progenitor{}/GCzsignchange_actions_snap_{}_jan19.txt'
        outplot = 'actions/right_potential/prog234_GCzsignchange_actions_snap_{}.png'
    elif excludedisk[jj]:
        infilename = 'actions/right_potential/progenitor{}/GCwodisk_actions_snap_{}_jan19.txt'
        outplot = 'actions/right_potential/prog234_GCwodisk_actions_snap_{}.png'
    else:
        infilename = 'actions/right_potential/progenitor{}/GCall_actions_snap_{}_jan19.txt'
        outplot = 'actions/right_potential/prog234_GCall_actions_snap_{}.png'

counter = 0
colors = ['black', 'black', 'xkcd:hot pink', 'xkcd:indigo blue', 'xkcd:saffron']
labels = ['\nJ$_\mathrm{R}$ [kpc km s$^{-1}$]\n', '\nL$_\mathrm{z}$ [kpc km s$^{-1}$]\n', '\nJ$_\mathrm{z}$ [kpc km s$^{-1}$]\n']

actionrange = ([0., 7500.], [-15000., 5000.], [0., 3500.])
for prognr in [4,3,2]:
    progcolor = colors[prognr]
    survivor_IDs, r_kpc, jR_kpckms, lz_kpckms, jz_kpckms = np.loadtxt(filedir + infilename.format(prognr, snapnr))
    data = np.vstack([jR_kpckms, lz_kpckms, jz_kpckms])
    if counter == 0:
        figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = progcolor, bbox_inches = 'tight', range = actionrange) #range = actionrange, 
        counter += 1
    else:         
        figure = corner.corner(data.transpose(), labels = labels, plot_contours = 1, color = progcolor, bbox_inches = 'tight', fig = figure, range = actionrange) #range = actionrange, 
figure.savefig(plotdir + outplot)


FileNotFoundError: [Errno 2] No such file or directory: '/u/milas/masterthesis/masterproject/files/actions/right_potential/progenitor4/GCall_actions_snap_127_jan19.txt'

In [ ]:
a